In [ ]:
import pymongo
import json
import os
import shutil
import config  # Import user-defined configuration

In [ ]:
def dbIngestion(json_directory, db_name):
    """
    Reads all JSON files from a directory and saves as denormalized documents in MongoDB.
    Parameters:
    - json_directory (str): Path to the directory containing JSON files.
    - db_name (str): Name of the MongoDB database.
    """
    try:
        # Validate that json_directory is a directory
        if not os.path.isdir(json_directory):
            raise NotADirectoryError(f"Provided path is not a directory: {json_directory}")

        # MongoDB connection
        client = pymongo.MongoClient(config.MONGO_URI)  # Connect to MongoDB
        db = client[db_name]  # Select database
        collection = db[config.COLLECTION_NAME]  # Select collection

        # Identify and load JSON files
        json_files = [os.path.join(json_directory, f) for f in os.listdir(json_directory) if f.endswith(".json")]
        if not json_files:
            print("No JSON files found in the directory.")
            return

        # Categorize files by keyword
        categorized_data = {
            "txparamsdab": [],
            "txantennadab": []
        }
        for json_file in json_files:
            filename = os.path.basename(json_file).lower()
            with open(json_file, "r") as file:
                file_data = json.load(file)
            if "txparamsdab" in filename:
                categorized_data["txparamsdab"] = file_data
            elif "txantennadab" in filename:
                categorized_data["txantennadab"] = file_data

        # Validate if all required data is loaded
        if not categorized_data["txparamsdab"] or not categorized_data["txantennadab"]:
            print("One or more required JSON files are missing.")
            return

        # Merge data based on the `id` field
        txparamsdab_data = {item["id"]: item for item in categorized_data["txparamsdab"]}
        txantennadab_data = {item["id"]: item for item in categorized_data["txantennadab"]}
        denormalized_data = []
        for record_id, txparams_record in txparamsdab_data.items():
            txantennadab_record = txantennadab_data.get(record_id, {})
            denormalized_document = {
                "id": record_id,
                "txparamsdab": {
                    "date": txparams_record.get("Date", ""),
                    "ensemble": txparams_record.get("Ensemble", ""),
                    "licence": txparams_record.get("Licence", ""),
                    "ensemble_area": txparams_record.get("Ensemble Area", ""),
                    "eid": txparams_record.get("EID", ""),
                    "transmitter_area": txparams_record.get("Transmitter Area", ""),
                    "site": txparams_record.get("Site", ""),
                    "frequency": txparams_record.get("Freq.", ""),
                    "block": txparams_record.get("Block", ""),
                    "tii_main_id_hex": txparams_record.get("TII Main Id (Hex)", ""),
                    "tii_sub_id_hex": txparams_record.get("TII Sub Id (Hex)", ""),
                    "service_labels": txparams_record.get("service_labels", [])
                },
                "txantennadab": {
                    "ngr": str(txantennadab_record.get("NGR", "")).strip(),
                    "site_height": str(txantennadab_record.get("Site Height", "")).strip(),
                    "latitude": str(txantennadab_record.get("latitude", "")).strip(),
                    "longitude": str(txantennadab_record.get("longitude", "")).strip(),
                    "in_use_ae_ht": str(txantennadab_record.get("in_use_ae_ht", "")),
                    "in_use_erp_total": str(txantennadab_record.get("in_use_erp_total", "")),
                    "dir_max_erp": str(txantennadab_record.get("dir_max_erp", "")),
                    "directional_data": txantennadab_record.get("directional_data", {})
                }
            }
            denormalized_data.append(denormalized_document)

        # Insert denormalized documents into MongoDB
        for document in denormalized_data:
            result = collection.insert_one(document)
            print(f"Inserted document with ID: {result.inserted_id}")

        # Backup processed JSON files
        backup_dir = os.path.join(json_directory, "json_backup")
        os.makedirs(backup_dir, exist_ok=True)
        for json_file in json_files:
            shutil.move(json_file, os.path.join(backup_dir, os.path.basename(json_file)))
            print(f"Moved {json_file} to {backup_dir}")
        print("Denormalized data ingestion completed successfully.")
    except NotADirectoryError as e:
        print(f"Error: {e}")
    except pymongo.errors.ConnectionError as e:
        print(f"MongoDB connection error: {e}")
    except Exception as e:
        print(f"Error during MongoDB ingestion: {e}")